### MARBL diagnostics notebook
#### Compares nutrients and O$_2$ to WOA
#### Also compares to another CESM simulation

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from glob import glob
from collections import OrderedDict
import xarray as xr
import numpy as np
import esmlab
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from scipy import stats
import pop_tools 
from statistics import mean
from matplotlib import style
from scipy import stats
style.use('ggplot')

In [2]:
case = 'g.e21.G1850ECOIAF.T62_g16.001'
path = '/glade/scratch/kristenk/archive/' + case + '/ocn/proc/tseries/month_1'
variables = ['NO3', 'PO4','AOU','O2','SiO3']
coords = {'x':'TLONG','y':'TLAT'}
keepthese = ['z_t','z_t_150m','time_bound','TAREA','REGION_MASK', 'time']

In [3]:
%%time

ds_list = []

print(f'reading {case}')

ds = xr.Dataset()

for v in variables:
    
    print(v)
    
    files = sorted(glob(f'{path}/{case}.pop.h.{v}.??????-??????.nc'))       

    #print(files)
    
    #dsv = xr.open_mfdataset(files, decode_times=True, decode_coords=False)
    dsv=xr.open_mfdataset(files, data_vars="minimal", coords='minimal', compat="override", parallel=True, concat_dim="time",
                       drop_variables=["transport_components", "transport_regions"], decode_times=True)

    
    ds = xr.merge((ds, dsv))
    ds_list.append(ds)

reading g.e21.G1850ECOIAF.T62_g16.001
NO3
PO4
AOU
O2
SiO3
CPU times: user 2.16 s, sys: 233 ms, total: 2.39 s
Wall time: 4.31 s


In [4]:
coords = {'x':'TLONG','y':'TLAT'}
keepthese=['z_t','z_t_150m','time','time_bound','TAREA','PO4','Fe','NO3','SiO3','AOU','O2','REGION_MASK']
keep_vars = keepthese +list(coords.values())+['dz','KMT']
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [ ]:
ds["time"] = ds.time_bound.compute().mean(dim="d2")

In [ ]:
NO3_diff = ds_annmean.NO3 - ds_woa.NO3
PO4_diff = ds_annmean.PO4 - ds_woa.PO4
SiO3_diff = ds_annmean.SiO3 - ds_woa.SiO3
AOU_diff = ds_annmean.AOU - ds_woa.AOU
O2_diff = ds_annmean.O2 - ds_woa.O2